In [126]:
from multiprocessing import Pool
from functools import partial
import numpy as np
#from numba import jit

In [127]:
#TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
        return score - true

    def h(self,true,score):
        return np.ones_like(score)

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
        return 1.0 / (1.0 + np.exp(-score))

    def g(self,true,score):
        return self.pred(score) - true

    def h(self,true,score):
        p = self.pred(score)
        return p * (1.0 - p)

In [186]:
# TODO: class of Random Forest
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

class RF(object):
    '''
    Class of Random Forest
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        rf = 0.99, num_trees = 80):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees
        
        self.rf_model = None
        if self.loss == 'mse':
            self.rf_model = RandomForestRegressor(
                n_estimators=self.num_trees,
                max_depth=self.max_depth,
                min_samples_split=self.min_sample_split,
                max_features=self.rf,
                n_jobs=self.n_threads
            )
        elif self.loss == 'log':
            self.rf_model = RandomForestClassifier(
                n_estimators=self.num_trees,
                max_depth=self.max_depth,
                min_samples_split=self.min_sample_split,
                max_features=self.rf,
                n_jobs=self.n_threads
            )
        else:
            raise ValueError("Invalid loss function.")


    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        self.rf_model.fit(train, target)
        return self

    def predict(self, test):
        #TODO
        return self.rf_model.predict(test)

In [187]:
# copied from homework 2
import pandas as pd
import numpy as np
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
feature = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
price = raw_df.values[1::2, 2]
print('data size = ', feature.shape)
print('target size = ', price.shape)

data size =  (506, 13)
target size =  (506,)


In [188]:
#RF and Boston dataset

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

features = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

y_train_pred_rf = rf_regressor.predict(X_train)
y_test_pred_rf = rf_regressor.predict(X_test)

rmse_train_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

# Print RMSE for training and test set
print("Random Forest Regression:")
print("Training RMSE:", rmse_train_rf)
print("Test RMSE:", rmse_test_rf)

Random Forest Regression:
Training RMSE: 1.4097371024484677
Test RMSE: 2.8415312704216915


In [189]:
#RF using RF class
features = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_custom = RF()
rf_custom.fit(X_train, y_train)

y_train_pred_rf = rf_custom.predict(X_train)
y_test_pred_rf = rf_custom.predict(X_test)

rmse_train_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

# Print RMSE for training and test sets
print("Random Forest Regression (Using RF class):")
print("Training RMSE:", rmse_train_rf)
print("Test RMSE:", rmse_test_rf)

Random Forest Regression (Using RF class):
Training RMSE: 3.409243104272463
Test RMSE: 3.6339459745688636


In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, price, test_size=0.3, random_state=8)

import numpy as np
#X= nxd matrix, y= n-dim vector

def least_square(X, y):
    theta= np.linalg.inv(X.T @ X) @ X.T @ y
    return theta

def ridge_reg(X, y, eta):
    d = X.shape[1]
    theta = np.linalg.inv(X.T @ X + eta * np.identity(d)) @ X.T @ y
    return theta

theta = least_square(X_train, y_train)
df_theta = pd.DataFrame(zip(feature, theta),columns=['Feature','Coeff'])

etas = [0.1, 1.0, 10.0, 100.0]
for eta in etas:
    theta_r = ridge_reg(X_train, y_train, eta)
    df_theta_r = pd.DataFrame(zip(feature, theta_r),columns=['Feature','Coeff'])
    
def pred_fn(X, theta):
    pred= X @ theta
    return pred

def root_mean_square_error(pred, y):
    rmse = np.sqrt(np.mean((y - pred)**2))
    return rmse

theta_linear = least_square(X_train, y_train)
theta_ridge = ridge_reg(X_train, y_train, 10.0)  # Example eta value

pred_linear_train = pred_fn(X_train, theta_linear)
pred_ridge_train = pred_fn(X_train, theta_ridge)

pred_linear_test = pred_fn(X_test, theta_linear)
pred_ridge_test = pred_fn(X_test, theta_ridge)

rmse_linear_train = root_mean_square_error(pred_linear_train, y_train)
rmse_ridge_train = root_mean_square_error(pred_ridge_train, y_train)

rmse_linear_test = root_mean_square_error(pred_linear_test, y_test)
rmse_ridge_test = root_mean_square_error(pred_ridge_test, y_test)

print("Linear Regression")
print("Training RMSE:", rmse_linear_train)
print("Test RMSE:", rmse_linear_test, '\n')

print("Ridge Regression")
print("Training RMSE:", rmse_ridge_train)
print("Test RMSE:", rmse_ridge_test)

Linear Regression
Training RMSE: 4.820626531838223
Test RMSE: 5.209217510530916 

Ridge Regression
Training RMSE: 4.829777333975097
Test RMSE: 5.189347305423606


In [133]:
#Credit g dataset
data_url = "https://www.openml.org/data/get_csv/31/dataset_31_credit-g.arff"
credit_data = pd.read_csv(data_url)

credit_data = pd.get_dummies(credit_data)

X = credit_data.drop(columns=['class_bad', 'class_good']) 
y = credit_data['class_good'] 

In [134]:
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Credit g accuracies")
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Credit g accuracies
Training Accuracy: 1.0
Test Accuracy: 0.79


In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RF(loss='log') 
rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Credit g accuracy using the RF class")
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Credit g accuracy using the RF class
Training Accuracy: 0.77
Test Accuracy: 0.755


In [136]:
# cancer dataset
data_path = "/Users/karolina/Desktop/Machine Learning/hw4/data.csv"  
cancer_data = pd.read_csv(data_path)

X = cancer_data.drop(columns=['diagnosis']) 
y = cancer_data['diagnosis']

In [137]:
#accuracy on cancer data
from sklearn.impute import SimpleImputer

# Impute missing values
X.drop(columns=['Unnamed: 32'], inplace=True)
imputer = SimpleImputer(strategy='mean') 
X_imputed = imputer.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_train_pred = rf_classifier.predict(X_train)
y_test_pred = rf_classifier.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Cancer diagnostic accuracies:")
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Cancer diagnostic accuracies:
Training Accuracy: 1.0
Test Accuracy: 0.9649122807017544


In [138]:
#cancer accuracy with RF class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RF(loss='log')
rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Cancer diagnostic accuracies with RF class')
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Cancer diagnostic accuracies with RF class
Training Accuracy: 0.9802197802197802
Test Accuracy: 0.956140350877193


In [192]:
# TODO: class of GBDT
from sklearn.tree import DecisionTreeRegressor 


class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'log',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        learning_rate = 0.1, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.trees = []

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        pred = np.zeros(len(target))

        for _ in range(self.num_trees):
            residuals = target - pred
            
            tree = DecisionTreeRegressor(max_depth=self.max_depth, 
                                         min_samples_split=self.min_sample_split)
            tree.fit(train, residuals)
            
            pred += self.learning_rate * tree.predict(train)
            self.trees.append(tree)

        return self
        

    def predict(self, test):
        #TODO
        pred = np.zeros(len(test))

        for tree in self.trees:
            pred += self.learning_rate * tree.predict(test)
        
        return pred

In [193]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.
    
    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.
    
    A node on a tree can be either a leaf node or a non-leaf node.
    '''
    
    #TODO
    def __init__(self, is_leaf=False, split_feature=None, split_threshold=None, left_child=None, right_child=None):
        self.is_leaf = is_leaf
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left
        # store essential information in every tree node
        
    

In [197]:
# TODO: class of single tree
from sklearn.tree import DecisionTreeRegressor 

class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''
    
    def __init__(self, n_threads = None, 
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0, rf = 0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.int_member = 0

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        #TODO
        self.root = self.construct_tree(train, g, h, self.max_depth)
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        #TODO
        
        pred = np.zeros(len(test))

        for tree in self.trees:
            pred += self.learning_rate * tree.predict(test)
        
        return pred

    def construct_tree(self, train, g, h, max_depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.
        
        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        #TODO
        
        if max_depth == 0 or len(train) < self.min_sample_split:
            return TreeNode(is_leaf=True)
        
        # Find the best decision rule for splitting
        feature, threshold, gain = self.find_best_decision_rule(train, g, h)
        
        if gain <= 0:
            return TreeNode(is_leaf=True)
        
        # Split the data based on the best decision rule
        left_train = train[train[:, feature] <= threshold]
        right_train = train[train[:, feature] > threshold]
        left_g = g[train[:, feature] <= threshold]
        right_g = g[train[:, feature] > threshold]
        left_h = h[train[:, feature] <= threshold]
        right_h = h[train[:, feature] > threshold]
        
        # Recursively construct left and right subtrees
        left_child = self.construct_tree(left_train, left_g, left_h, max_depth - 1)
        right_child = self.construct_tree(right_train, right_g, right_h, max_depth - 1)
        
        # Construct the current node
        return TreeNode(split_feature=feature, split_threshold=threshold, 
                        left_child=left_child, right_child=right_child)

    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j, 
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train
        
        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        #TODO
        num_features = train.shape[1]

        def find_best_decision_rule_for_feature(feature):
            threshold, gain = self.find_threshold(g, h, train[:, feature])
            return feature, threshold, gain

        with Pool(processes=self.n_threads) as pool:
            results = pool.map(find_best_decision_rule_for_feature, range(num_features))

        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        for feature, threshold, gain in results:
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        return best_feature, best_threshold, best_gain

    
    def find_threshold(self, g, h, train):
        '''
        Given a particular feature $p_j$,
        return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        #TODO               
        sorted_indices = np.argsort(train)
        train_sorted = train[sorted_indices]
        g_sorted = g[sorted_indices]
        h_sorted = h[sorted_indices]

        best_gain = -np.inf
        best_threshold = None

        sum_g = 0
        sum_h = 0

        for i in range(len(train_sorted) - 1):
            sum_g += g_sorted[i]
            sum_h += h_sorted[i]

            # If the current and next feature values are not equal, compute gain
            if train_sorted[i] != train_sorted[i + 1]:
                gain = 0.5 * ((sum_g ** 2) / (sum_h + self.lamda) + sum_h * self.gamma)
                if gain > best_gain:
                    best_gain = gain
                    best_threshold = 0.5 * (train_sorted[i] + train_sorted[i + 1])

        return best_threshold, best_gain

In [212]:
# TODO: GBDT regression on boston house price dataset
# load data
import numpy as np
import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(506, 13) (506,) (354, 13) (354,) (152, 13) (152,)


In [213]:
gbdt = GBDT(loss=leastsquare(), max_depth=3, min_sample_split=10, lamda=1, gamma=0, learning_rate=0.1, num_trees=100)
gbdt.fit(X_train, y_train)

y_train_pred = gbdt.predict(X_train)
y_test_pred = gbdt.predict(X_test)

train_rmse = sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = sqrt(mean_squared_error(y_test, y_test_pred))

print("Training RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

Training RMSE: 1.309516746094146
Test RMSE: 3.499149031094111


In [219]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/', parser= 'auto')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1000, 20) (1000,) (700, 20) (700,) (300, 20) (300,)


In [220]:
# TODO: GBDT classification on credit-g dataset
# load data

from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/', parser='auto', as_frame=False)

# Convert target variable to binary numeric values
y = np.array([1 if label == 'good' else 0 for label in y])

# Convert text data to numerical
label_encoder = LabelEncoder()
for column in range(X.shape[1]):
    if X[:, column].dtype == 'object':
        X[:, column] = label_encoder.fit_transform(X[:, column])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Instantiate the GBDT model
gbdt = GBDT(max_depth=3, min_sample_split=10, learning_rate=0.1, num_trees=100)

# Fit the model on the training data
gbdt.fit(X_train, y_train)

# Make predictions on training and test data
train_predictions = gbdt.predict(X_train)
test_predictions = gbdt.predict(X_test)

# Calculate training accuracy
train_accuracy = np.mean((train_predictions >= 0.5) == y_train)

# Calculate test accuracy
test_accuracy = np.mean((test_predictions >= 0.5) == y_test)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

(1000, 20) (1000,) (700, 20) (700,) (300, 20) (300,)
Training Accuracy: 0.9128571428571428
Test Accuracy: 0.75


In [221]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(569, 30) (569,) (398, 30) (398,) (171, 30) (171,)


In [222]:
#gbdt on cancer
gbdt_model = GBDT(loss=logistic(), max_depth=4, min_sample_split=10, lamda=1, gamma=0, learning_rate=0.1, num_trees=100)

gbdt_model.fit(X_train, y_train)

train_probability =gbdt_model.predict(X_train)
test_probability = gbdt_model.predict(X_test)

train_predictions = np.where(train_probability >= 0.5, 1, 0)
test_predictions = np.where(test_probability >= 0.5, 1, 0)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 1.0
Test Accuracy: 0.9649122807017544
